# Importing the libraries

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

# Setting Image and Batch Size

In [9]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

# Loading the train, val, and test data

In [10]:
train = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


# Adding augmentation layer

In [11]:
# Using this method for M1 Mac CPU acces to run augmentation
with tf.device('/cpu:0'):
  data_augmentation = keras.Sequential(
  [
      layers.RandomFlip("horizontal"),
      layers.RandomRotation(0.1),
      layers.RandomZoom(0.2)
      # layers.RandomContrast(0.2),
      # layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
  ] 
  )

# Defining the model

In [13]:
inputs = keras.Input(shape=(100, 100, 3))
x = inputs
with tf.device('/cpu:0'):
  x = data_augmentation(x)
x = layers.Rescaling(1./255)(x)
x = data_augmentation(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(6, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential_1 (Sequential)      (None, 100, 100, 3)  0           ['input_3[0][0]',                
                                                                  'rescaling_2[0][0]']            
                                                                                                  
 rescaling_2 (Rescaling)        (None, 100, 100, 3)  0           ['sequential_1[2][0]']           
                                                                                            

# Adding Callbacks

In [14]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath="../../models/zenodo/m4_Augmentation.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

# Running the model

In [15]:
history = model.fit(
  train.cache(),
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/40


2022-08-03 15:19:28.488875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 1.7924 - accuracy: 0.1807

2022-08-03 15:19:36.398141: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 9s 75ms/step - loss: 1.7924 - accuracy: 0.1807 - val_loss: 2.3359 - val_accuracy: 0.1667
Epoch 2/40
105/105 [==============================] - 8s 72ms/step - loss: 1.6759 - accuracy: 0.2905 - val_loss: 2.1983 - val_accuracy: 0.3444
Epoch 3/40
105/105 [==============================] - 7s 70ms/step - loss: 1.5577 - accuracy: 0.3688 - val_loss: 3.5789 - val_accuracy: 0.2153
Epoch 4/40
105/105 [==============================] - 7s 70ms/step - loss: 1.4881 - accuracy: 0.4164 - val_loss: 3.8642 - val_accuracy: 0.2347
Epoch 5/40
105/105 [==============================] - 7s 70ms/step - loss: 1.3767 - accuracy: 0.4583 - val_loss: 3.8515 - val_accuracy: 0.2764
Epoch 6/40
105/105 [==============================] - 7s 71ms/step - loss: 1.2963 - accuracy: 0.4866 - val_loss: 1.6292 - val_accuracy: 0.4556
Epoch 7/40
105/105 [==============================] - 7s 70ms/step - loss: 1.2340 - accuracy: 0.5170 - val_loss: 2.3514 - val_accuracy: 0.4014
Epoch 8/40

# Evaluating the model

In [16]:

model.evaluate(test)

load_model = keras.models.load_model("../../models/zenodo/m4_Augmentation.h5")
load_model.evaluate(test)

 4/23 [====>.........................] - ETA: 0s - loss: 0.8061 - accuracy: 0.8281

2022-08-03 15:31:16.094795: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 0s 17ms/step - loss: 0.8682 - accuracy: 0.7625


[0.8682317137718201, 0.7625000476837158]